In [1]:
import glob
import cv2
import os.path as path
import numpy as np

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from common import *

%matplotlib inline


In [2]:
def extract_features(imgs, color_space='RGB', spatial_size=(32, 32),
                        hist_bins=32, orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0,
                        spatial_feat=True, hist_feat=True, hog_feat=True):
    features = []
    
    for image in imgs:
        file_features = []
        feature_image = convert_colorspace(image, color_space)
        
        if spatial_feat == True:
            spatial_features = bin_spatial(feature_image, size=spatial_size)
            file_features.append(spatial_features)
        if hist_feat == True:
            hist_features = color_hist(feature_image, nbins=hist_bins)
            file_features.append(hist_features)
        if hog_feat == True:
            if hog_channel == 'ALL':
                hog_features = []
                for channel in range(feature_image.shape[2]):
                    hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                        orient, pix_per_cell, cell_per_block, 
                                        vis=False, feature_vec=True))
                hog_features = np.ravel(hog_features)        
            else:
                hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                            pix_per_cell, cell_per_block, vis=False, feature_vec=True)
                
            file_features.append(hog_features)
        features.append(np.concatenate(file_features))
    return features


def read_dataset(data_path):
    pattern = '**/*.png'
    files  = glob.glob(path.join(data_path, pattern))
    return [read_image(fn) for fn in files]



# Preparing the dataset

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [4]:
cars = read_dataset('./data/vehicles')
non_cars = read_dataset('./data/non-vehicles')

print("Cars dataset size: ", len(cars))
print("Non-cars dataset size: ", len(non_cars))

Cars dataset size:  8792
Non-cars dataset size:  8968


In [5]:
X = np.concatenate((cars, non_cars))
y = np.concatenate([np.ones(len(cars)), np.zeros(len(non_cars))])

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print("Samples in training dataset: ", len(X_train))
print("Samples in test dataset: ", len(X_test))


Samples in training dataset:  14208
Samples in test dataset:  3552


# Train SVM classifier

In [7]:
from sklearn.model_selection import GridSearchCV, ShuffleSplit, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

class FeatureExtractor:
    def __init__(self, **params):
        self.params = params
        
    def fit(self, X, y):
        return self
    
    def transform(self, X):
        return extract_features(X, 
                                color_space=self.params['color_space'], 
                                hog_channel=self.params['hog_channel'],
                                orient=self.params['orient'],
                                pix_per_cell = self.params['pix_per_cell'],
                                cell_per_block = self.params['cell_per_block'],
                                spatial_feat=self.params['spatial_size'] > 0, 
                                spatial_size=(self.params['spatial_size'], self.params['spatial_size']),
                                hist_bins=(self.params['hist_bins']),
                                hist_feat=self.params['hist_bins'] > 0)
    
    def get_params(self, deep=False):
        return self.params
    
    def set_params(self, **params):
        self.params.update(params)
        
classifier = Pipeline([('features', FeatureExtractor()),
                       ('scaler', StandardScaler()),
                       ('svc', LinearSVC())])

param_grid = {
    'features__color_space': ['RGB', 'HLS','YUV', 'YCrCb'],
    'features__hog_channel': ['ALL'],
    'features__orient': [7, 9, 11],
    'features__hist_bins': [0, 2, 4, 8, 16, 32, 64],
    'features__spatial_size': [0, 4, 8, 16, 32],
    'features__pix_per_cell': [8],
    'features__cell_per_block': [1, 2, 4],
    'svc__C': [0.0001, 0.001, 0.01, 1, 10]
}

search = RandomizedSearchCV(classifier, param_grid, 
                            cv=ShuffleSplit(test_size=0.2, n_splits=1),
                            n_iter=20)
search.fit(X_train, y_train)

RandomizedSearchCV(cv=ShuffleSplit(n_splits=1, random_state=None, test_size=0.2, train_size=None),
          error_score='raise',
          estimator=Pipeline(steps=[('features', <__main__.FeatureExtractor object at 0x11ada0588>), ('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('svc', LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0))]),
          fit_params={}, iid=True, n_iter=20, n_jobs=1,
          param_distributions={'features__orient': [7, 9, 11], 'features__color_space': ['RGB', 'HLS', 'YUV', 'YCrCb'], 'features__hog_channel': ['ALL'], 'features__hist_bins': [0, 2, 4, 8, 16, 32, 64], 'features__pix_per_cell': [8], 'svc__C': [0.0001, 0.001, 0.01, 1, 10], 'features__cell_per_block': [1, 2, 4], 'features__spatial_size': [0, 4, 8, 16, 32]},
          pre_dispatch='2*n_jobs', random_state=None, ref

In [8]:
import pandas as pd
results = pd.DataFrame(search.cv_results_)
results.sort_values(by='rank_test_score', inplace=True)
results.to_csv('./output_model/gridsearch_result.csv')

In [9]:
search.best_params_

{'features__cell_per_block': 2,
 'features__color_space': 'YUV',
 'features__hist_bins': 32,
 'features__hog_channel': 'ALL',
 'features__orient': 11,
 'features__pix_per_cell': 8,
 'features__spatial_size': 32,
 'svc__C': 0.01}

In [10]:
final_classifier = search.best_estimator_

In [11]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

y_pred = final_classifier.predict(X_test)
print(classification_report(y_test, y_pred, target_names=['Non-cars', 'Cars']))
print("Confusion matrix: ")
print(confusion_matrix(y_test, y_pred))
print("Final classifier score: %.4f" % accuracy_score(y_test, y_pred))


             precision    recall  f1-score   support

   Non-cars       0.99      1.00      1.00      1829
       Cars       1.00      0.99      0.99      1723

avg / total       0.99      0.99      0.99      3552

Confusion matrix: 
[[1821    8]
 [  10 1713]]
Final classifier score: 0.9949


# Save classifier and parameters

In [12]:
from sklearn.externals import joblib

feature_params = final_classifier.named_steps['features'].params
raw_classifier = Pipeline(final_classifier.steps[1:])

data = {
    'feature_params': feature_params,
    'classifier': raw_classifier
}

joblib.dump(data, './output_model/model.pkl')

['./output_model/model.pkl']